# Mapping_manf_to_Retailer

Installing required packages

In [1]:
!pip install recordlinkage

  Created wheel for jellyfish: filename=jellyfish-0.8.2-py3-none-any.whl size=10424 sha256=433367ac0fa5d39cc43815a3e71b4bd572a2de6c0cdb2ff760aebf62d8ba12ad
  Stored in directory: c:\users\sairam\appdata\local\pip\cache\wheels\12\9e\c1\46af1db49a60a4adf555196c0ead132e4a36a2bd69b078e740
Successfully built jellyfish


In [10]:
import pandas as pd
import recordlinkage

In [11]:
df_m = pd.read_excel('data/manuf_data.xlsx') # reading of manfacturing data
df_r = pd.read_excel('data/retailer_data.xlsx')# reading of reatiler data

df_m.set_index('prod_id',inplace=True) # making product id as index
df_r.set_index('ret_prod_id',inplace=True) # making product id as index

In [12]:
df_m_dict = df_m.to_dict('dict')['Description'] #dictionary shoud be made so that it need to map after comparison
df_r_dict = df_r.to_dict('dict')['Product_Description']#both manfacturing and retailer

In [13]:
def get_top_four_probabilities(prob,**kwargs): #function to get top four better probabilities for string comparison
    top = kwargs.get('top',4)
    initial=True
    manf_prod_id = prob['prod_id'].unique()
    for each_id in manf_prod_id:
        temp = prob[prob['prod_id']==each_id]
        temp = temp.sort_values(by='probability',ascending=False)[:top] #sorting based on probability
        if initial:
            result = temp
            initial = False
        else:
            result = result.append(temp)   
    result.reset_index(drop=True,inplace=True)
    return result

In [6]:
# initilasing record linkage algorthim for effective search for de deuplication and comaparison of strings
indexer = recordlinkage.Index()
indexer.full()
index_links = indexer.index(df_m, df_r) # need to pass dataframes to be compared in order make index pairs

comp = recordlinkage.Compare() #initilasing compare algo and passing jarowinkler distance calculation string comparison a effective for string comparison
comp.string('Description', 'Product_Description', method='jarowinkler',label='probability')

prob=comp.compute(index_links, df_m, df_r) # computing the distance and rersults are saved in prob
prob = prob.reset_index() 
result = get_top_four_probabilities(prob)

In [7]:
# A proper index should be replaced with corresponding dictinary pairs in order to make readable
result['manf_product_description']  =result['prod_id'].replace(df_m_dict)
result['retailer_product_description']  =result['ret_prod_id'].replace(df_r_dict)
end_result = result[['prod_id','manf_product_description','probability','retailer_product_description','ret_prod_id']]
end_result.to_csv('mapping_results.csv') #preparing columns for reader comfort and saving to a csv 

In [8]:
result

,prod_id,ret_prod_id,probability,manf_product_description,retailer_product_description
0,355,599,0.917836,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Repair Shampoo 250ml
1,355,1379,0.864511,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Body Lotion 400ml
2,355,780,0.852924,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Body Lotion 400ml
3,355,351,0.850166,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Pro Age Shampoo 400ml
4,356,599,0.917836,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Repair Shampoo 250ml
...,...,...,...,...,...
1539,2608,94,0.619118,Cbv Inco Rinse Off Scrub 300ml,Cbvery Remoisturising Conditioner 900ml
1540,2644,341,0.838458,Fiur Gum Stick Tin 75ml,Fiur Texturising Fibre 75ml
1541,2644,347,0.820797,Fiur Gum Stick Tin 75ml,Fiur Men Mattifying Putty 75ml
1542,2644,96,0.697970,Fiur Gum Stick Tin 75ml,Fiur Radiating Tropical Oil 50ml


In [23]:
result.head(4) #mapping between top 4 descriptions

,prod_id,ret_prod_id,probability,manf_product_description,retailer_product_description
0,355,599,0.917836,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Repair Shampoo 250ml
1,355,1379,0.864511,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Body Lotion 400ml
2,355,780,0.852924,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Intensive Body Lotion 400ml
3,355,351,0.850166,Mexifiz Shampoo Intensive Repair 400ml,Mexifiz Pro Age Shampoo 400ml


In [19]:
result.manf_product_description.unique() #unique manufacturing product descriptions

array(['Mexifiz Shampoo Intensive Repair 400ml',
       'Cbv Shampoo Moisture Rich 900ml',
       'Cbvery Hairspray Extra Hold 400ml',
       'Oddaced Paste Extreme Matt Clay Tin 75ml',
       'Cbv Co Moisture Rich Bot 900ml',
       'Cbv Heat Protection Spr Heat Def 300ml',
       'Cbvery Hairspray Freeze Hold 400ml',
       'Cbv Shampoo Moisture Rich 500ml',
       'Mexifiz Co Intensive Bot 350ml',
       'Oddaced Wax Rework Putty Jar 150ml',
       'Mexifiz Shampoo Daily Care 400ml',
       'Cbv Co Moisture Rich Bot 500ml',
       'Oddaced Gel Mega Hold Tube 200ml',
       'Mexifiz Men Shampoo Clean 250ml',
       'Oddaced Gel Spray Mega Hold 200ml',
       'Sinoyez Cream Protein Enriched 250ml',
       'Oddaced Gum Texturising Tin 75ml',
       'Oddaced Leave in Spray Heat Protect 200ml',
       'Cbv Shampoo Deep Cleansing 900ml',
       '6Good Shampoo Gentle Care Bot 200ml', 'Cbv Shampoo Keratin 400ml',
       'Cbv Shampoo Moisture Rich 800ml',
       'Cbv Shampoo Colour Revitalis

In [20]:
result.manf_product_description.value_counts()  #unique manufacturing product descriptions count

Cbv Shampoo Healthy Volume 235ml         16
Cbv Shampoo Moisture Rich 500ml          16
Cbv Co Keratin Bot 700ml                 16
Cbv Co Volume Bot 235ml                  16
Cbv Shampoo Deep Cleansing 500ml         16
                                         ..
Cbv Shampoo Cleanse & Replenish 235ml     4
Fiur Lotion Volume 236ml                  4
Mexifiz Co Strengthening Bot 200ml        4
6Good Shampoo Gentle Care Bot 200ml       4
Fiur Co Fine Hair 250ml                   4
Name: manf_product_description, Length: 291, dtype: int64

In [21]:
result.retailer_product_description.unique()   #unique retailer product descriptions

array(['Mexifiz Intensive Repair Shampoo 250ml',
       'Mexifiz Intensive Body Lotion 400ml',
       'Mexifiz  Intensive Body Lotion 400ml',
       'Mexifiz Pro Age Shampoo 400ml',
       'Cbvery Silky Smooth Conditioner 900ml',
       'Cbvery Remoisturising Conditioner 900ml',
       'Cbvery Silky Smooth Conditioner 500ml',
       'Cbvery Moisture Rich Shampoo 1.3ml',
       'Cbvery Extra Hold Hairspray 400ml',
       'Cbvery Firm Hold Hairspray 400ml',
       'Cbvery Freeze Hold Hairspray 400ml',
       'Cbvery Ultimate Hold Hairspray 400ml', 'Oddaced Fluff Tamer 75ml',
       'Oddaced Matt Fibre 75ml', 'Oddaced SurfStyle Paste 150ml',
       'Oddaced Ultimate Hold Hairspray 400ml',
       'Cbvery Colour Revitalise Shampoo 900 ml',
       'Cbvery Remoisturising Conditioner 500ml',
       'Cbvery Moisture Rich Conditioner 1.3ml',
       'Cbvery Pro Collection Beauty-Full Volume Shampoo 700ml',
       'Cbvery Pro Collection Keratin Smooth Conditioner 400ml',
       'Cbvery Pro Collect

In [22]:
result.retailer_product_description.value_counts() #unique retailer product descriptions count

Cbvery Silky Smooth Conditioner 500ml                                                              77
Cbvery Biotin+ Repair 7 Conditioner 400ml                                                          63
Cbvery Silky Smooth Conditioner 900ml                                                              62
Cbvery Colour Revitalise Shampoo 900 ml                                                            49
Cbvery Biotin+ Repair 7 Shampoo 400ml                                                              37
                                                                                                   ..
Twicebeaut Muru Muru Butter & Rose Blooming Colour Vegan Shampoo for colour treated hair 400 ml     1
Parrot  Natural Casual Styling Clay 75ml                                                            1
Cbvery Pro Collection Botanique Nourish & Replenish Shampoo 700ml                                   1
Fiur Flawless Sleek Creation Hairspray 250ml                                      

When compared to NLTK, recordlinkage works efficiently and gives better results